### 1.　J-Quants API にアクセス

In [ ]:
pip install jquants-api-client

#### 1.0　データ取得の流れ

1. メールアドレスとパスワードを用いてWeb画面かAPIでリフレッシュトークンを取得する。
2. リフレッシュトークンを用いて専用APIでIDトークンを取得する。
3. IDトークンを用いて、各種情報を取得するAPIを利用する。

#### 1.1　tokenの取得

##### 1.1.1　リフレッシュトークンの取得

In [3]:
import json
import requests
import pandas as pd


# POSTするデータを作る。
account_data = json.dumps({
        "mailaddress": "your-email",
        "password": "your-pass",
    })

auth_user_url = "https://api.jquants.com/v1/token/auth_user"

auth_result = requests.post(auth_user_url, data=account_data)
refresh_token = auth_result.json()["refreshToken"]

##### 1.1.2　IDトークンの取得

In [4]:
auth_refresh_url=f"https://api.jquants.com/v1/token/auth_refresh?refreshtoken={refresh_token}"
refresh_result = requests.post(auth_refresh_url)
id_token = refresh_result.json()["idToken"]

### 2. 各種データの取得

#### 2.1　上場銘柄一覧

In [5]:
import requests
import json

idToken = id_token
headers = {'Authorization': 'Bearer {}'.format(idToken)}
company = requests.get("https://api.jquants.com/v1/listed/info", headers=headers)

In [6]:
company_df = pd.DataFrame(company.json()['info'])
company_df.head()

,Date,Code,CompanyName,CompanyNameEnglish,Sector17Code,Sector17CodeName,Sector33Code,Sector33CodeName,ScaleCategory,MarketCode,MarketCodeName
0,2023-02-10,13010,極洋,"KYOKUYO CO.,LTD.",1,食品,0050,水産・農林業,TOPIX Small 2,0111,プライム
1,2023-02-10,13050,大和アセットマネジメント株式会社 ダイワ上場投信−トピックス,Daiwa ETF-TOPIX,99,その他,9999,その他,-,0109,その他
2,2023-02-10,13060,野村アセットマネジメント株式会社 ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,NEXT FUNDS TOPIX Exchange Traded Fund,99,その他,9999,その他,-,0109,その他
3,2023-02-10,13080,日興アセットマネジメント株式会社 上場インデックスファンドＴＯＰＩＸ,Nikko Exchange Traded Index Fund TOPIX,99,その他,9999,その他,-,0109,その他
4,2023-02-10,13090,野村アセットマネジメント株式会社 ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０...,NEXT FUNDS ChinaAMC SSE50 Index Exchange Trade...,99,その他,9999,その他,-,0109,その他


#### 2.1　株価四本値（/prices/daily_quotes）

In [7]:
import requests
import json

idToken = id_token
headers = {'Authorization': 'Bearer {}'.format(idToken)}
daily_quotes = requests.get("https://api.jquants.com/v1/prices/daily_quotes?&code=13010&from=20220101&to=20221231", headers=headers)

In [8]:
daily_quotes_df = pd.DataFrame(daily_quotes.json()['daily_quotes'])

In [9]:
daily_quotes_df.head()

,Date,Code,Open,High,Low,Close,Volume,TurnoverValue,AdjustmentFactor,AdjustmentOpen,AdjustmentHigh,AdjustmentLow,AdjustmentClose,AdjustmentVolume
0,2022-01-04,13010,3110.0,3110.0,3075.0,3085.0,9200.0,28436000.0,1.0,3110.0,3110.0,3075.0,3085.0,9200.0
1,2022-01-05,13010,3100.0,3115.0,3075.0,3110.0,6900.0,21394000.0,1.0,3100.0,3115.0,3075.0,3110.0,6900.0
2,2022-01-06,13010,3085.0,3115.0,3075.0,3075.0,9500.0,29362500.0,1.0,3085.0,3115.0,3075.0,3075.0,9500.0
3,2022-01-07,13010,3080.0,3115.0,3070.0,3070.0,11300.0,34835000.0,1.0,3080.0,3115.0,3070.0,3070.0,11300.0
4,2022-01-11,13010,3070.0,3090.0,3055.0,3085.0,10500.0,32281500.0,1.0,3070.0,3090.0,3055.0,3085.0,10500.0
